## Exercise 1:
In class we have gone through a rule-based NLP pipeline by executing components one by one. Typically, we create a pipeline as a single class that links all modules.

In this exercise, you will need to write a pipeline class. Let's call it **MyPipe**, it can be initiated through taking a set of rules. After that, it can be called through a **process** function, which can take a document text, and output a set of values: 
- document level classification
- context document, which is an object of type pyConTextGraph.ConTextDocument;
- annotations (in dataframe format);
- relations (in dataframe format).

In [1]:
# state your import here

from PyRuSH.RuSH import RuSH
from pyConTextNLP import pyConTextGraph
from pyConTextNLP.utils import get_document_markups


from DocumentClassifier import FeatureInferencer
from DocumentClassifier import DocumentInferencer
from nlp_pneumonia_utils import markup_sentence
from itemData import get_item_data
from visual import convertMarkups2DF


# begin to define MyPipe class
class MyPipe:
    def __init__(self, sentence_rules, target_rules, context_rules, feature_inference_rule, document_inference_rule):
        # initiate necessary components here
        self.sentence_rules = sentence_rules
        self.target_rules = target_rules
        self.context_rules = context_rules
        self.feature_inference_rule = feature_inference_rule
        self.document_inference_rule = document_inference_rule
        
        pass
    
    def process(self, doc_text):        
        #process your input doc_text, return the required results
        self.doc_text = doc_text
        
        
        
        return doc_class, context_doc, annotations, relations

Once your pipeline class is defined, you can use it multiple times for different set of rules.

## Exercise 2:
Now you can select documents from the MIMIC database (limit to 5 documents that contain your target concept), write a script to process all of them, and output a dictionary which uses document name as keys and document level classification as values.

In [22]:
import pymysql       #imports the Python mysql module
import pandas as pd  #imports the Python data analysis library 
import getpass       #imports the getpass module
from nltk.tokenize import sent_tokenize       #imports a sentence splitter
from nltk.tokenize import word_tokenize       #imports a string tokenizer breaking on whitespace
from nltk.tokenize import wordpunct_tokenize  #imports a string tokenizer breaking on whitespace and punctuation
from nltk.tokenize import WhitespaceTokenizer #used for generating word spans later
from nltk import pos_tag                      #a part-of-speech tagger
import re                                     #imports Python's regular expression functions


In [3]:
dbconn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for user jovyan"),db='mimic2')
cursor = dbconn.cursor()

Enter MySQL passwd for user jovyan········


In [ ]:
tables = pd.read_sql("SELECT table_name FROM information_schema.tables where table_schema='mimic2'", dbconn)
#print(tables)
print(pd.read_sql("SELECT count(text) from noteevents WHERE text like '%HIV%' LIMIT 5",dbconn))

In [16]:
num_notes = cursor.execute("SELECT text from noteevents WHERE text like '%HIV%' limit 5")
print("Read", num_notes,"notes from noteevents.\n")
note_list = []
for note in cursor:                   #grab each note from the SELECT results        
    note_list.append(str(note))       #add each new note to a list of notes as a string

Read 5 notes from noteevents.



In [60]:
sentences = sent_tokenize(note_list[0])
i = 0
for sentence in sentences:
    print (i,":", sentence, "\n")
    i+= 1

0 : ('\n\nAdmission Date:  [**2778-3-24**]       Discharge Date:  [**2778-3-27**]\n\nDate of Birth:   [**2729-8-8**]       Sex:  M\n\nService:  [**Company **] EAST\n\nCHIEF COMPLAINT:  Lethargy.\n\nHISTORY OF PRESENT ILLNESS:  This is a 48-year-old HIV\npositive African-American male with a history of\npolysubstance abuse, hepatitis C, cirrhosis and hypertension\nwho was admitted to [**Hospital 10736**] Hospital on [**2778-3-20**], for\ndetox following a recent binge using crack cocaine (about one\ngram per day), alcohol (one pint of wine per day) and Xanax\nand Klonopin while on Methadone maintenance. 

1 : On [**2778-3-22**], he was noted to have fluctuating mental status\n(somnolence, disorientation) and after being found down\nbeside his bed confused, with a bloody nose and a blood\npressure of 170/130, he was presumed to have had a seizure.\nHe was given 1 mg of Ativan and was brought to the [**Hospital1 184**] Emergency Department.\nFollowing a negative head CT, no further seizur

In [61]:
# configure your rules 
sentence_rules='KB/rush_rules.tsv'
target_rules='''
Comments: ''
Direction: ''
Lex: hiv
Regex: ''
Type: HIV
---
Comments: ''
Direction: ''
Lex: positive hiv
Regex: '1\d\d\.\d H'
Type: HIV'''
context_rules='''
Comments: ''
Direction: forward
Lex: 'no'
Regex: ''
Type: DEFINITE_NEGATED_EXISTENCE
---
Comments: ''
Direction: forward
Lex: 'denies'
Regex: ''
Type: DEFINITE_NEGATED_EXISTENCE
'''
feature_inference_rule='''
#Conclusion type, Evidence type, Modifier values associated with the evidence
NEGATED_CONCEPT,HIV,DEFINITE_NEGATED_EXISTENCE
'''
document_inference_rule='''
#Conclusion Type at document level, Evidence type at mention level
HIV_DOC,HIV

#Default document type
NO_HIV
'''
sentence_segmenter = RuSH(sentence_rules)
feature_inferencer=FeatureInferencer(feature_inference_rule)
document_inferencer = DocumentInferencer(document_inference_rule)

targets=get_item_data(target_rules)
modifiers=get_item_data(context_rules)
#print(sentence_segmenter)

In [62]:
#initiate an instance of MyPipe
myPipe=MyPipe(sentence_rules, target_rules, context_rules, feature_inference_rule, document_inference_rule)

In [63]:
sentences=sentence_segmenter.segToSentenceSpans(note_list[2])

ValueError: IntervalTree: Null Interval objects not allowed in IntervalTree: Interval(1332, 1331, 31)

In [59]:
for sentence in sentences:
    print("Sentence({}-{}):\t{}".format(sentence.begin, sentence.end, note_list[2][sentence.begin:sentence.end]))
    print('\n'+'-'* 100+'\n')

AttributeError: 'str' object has no attribute 'begin'

In [44]:
# initiate a pyConTextGraph to hold the pyConText output
context_doc = pyConTextGraph.ConTextDocument()

for sentence in sentences:
    sentence_text=note_list[1][sentence.begin:sentence.end].lower()
    # Process every sentence by adding markup
    m = markup_sentence(sentence_text, modifiers=modifiers, targets=targets)
    context_doc.addMarkup(m)
    context_doc.getSectionMarkups()
    print(m)


__________________________________________
rawText: ("
cleanedText: ("
__________________________________________

__________________________________________
rawText: n     date: [**2778-3-24**] 12:55 pm\n     chest (portable ap)                                             clip # [**clip number (radiology) 1896
cleanedText: n date: [**2778-3-24**] 12:55 pm\n chest (portable ap) clip # [**clip number (radiology) 1896
__________________________________________

__________________________________________
rawText: reason: r/o pna                                                     \n     ______________________________________________________________________________\n     underlying medical condition:\n      48 year old man with hiv admitted with ms changes, likely overdose, but can't \n      r/o infection.\n
cleanedText: reason: r/o pna \n ______________________________________________________________________________\n underlying medical condition:\n 48 year old man with hiv admitted with 

In [45]:
# convert graphic markups into dataframe    
markups = get_document_markups(context_doc)
annotations, relations, doc_txt = convertMarkups2DF(markups) 

In [46]:
display(annotations)
display(relations)

,markup_id,vis_category,start,end,txt,type
0,T0,Target,153.0,156.0,hiv,hiv
1,T1,Target,416.0,419.0,hiv,hiv


,relation_id,type,arg1_cate,arg1_id,arg2_cate,arg2_id


In [47]:
# apply inferences for document classication
inferenced_types = feature_inferencer.process(annotations, relations)
print('After inferred from modifier values, we got these types:\n '+str(inferenced_types))
doc_class = document_inferencer.process(inferenced_types)
print('\nDocument classification: '+ doc_class )

After inferred from modifier values, we got these types:
 ['hiv', 'hiv']

Document classification: hiv_doc


In [48]:
# write you code here, fill the results by processing each document through MyPipe
results=dict()  # this dictionary will contain document names as keys and a document-level classification as values.



## Exercise 3:

Now you get the results, but how can you be sure if they are correct? Let's dig a little deeper to visualize them. 

Hint: **view_pycontext_output** can take in a list of ConTextDocuments and visualize them.


In [37]:
from visual import view_pycontext_output
from visual import Vis

In [7]:
#get the context documents that you created in Exercise 2.
context_docs=[]



Change the file name to contain your UNID.

In [39]:
view_pycontext_output(context_doc, Vis(file_name="u1166466.html"))

***Please make a screenshot of the visualization file and submit it as homework assignment.***